In [30]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers

In [31]:
# turn list of lists of equivalent braids with corresponding answers into trainable data
def parse_equivalent_braids(allBraids, answers, maxStrands, maxCrossings):

    x_data = []
    y_data = []

    # zip list of equivalent braids with same label
    for equivalentBraids, answer in zip(allBraids, answers):
        for braid in equivalentBraids:
            x_data.append(braid)
            y_data.append(answer)

    # encode each braid word
    x_data = [encode_braid(x, maxStrands, maxCrossings) for x in x_data]

    return np.asarray(x_data), np.asarray(y_data)

In [32]:
# one-hot encode braid word and pad to consistent length
def encode_braid(braid, maxStrands, maxCrossings):

    b = tf.keras.utils.to_categorical(braid, num_classes=maxStrands*2)

    for i in range(maxCrossings - len(b)):
            b = np.append(b, [np.zeros(maxStrands*2)], axis=0)

    return b

In [33]:
# read pre-parsed knot data with braid words and equivalent braids

knot_data = pd.read_csv('../data/knot_data_jones.csv')
knot_data["Equivalent Braids"] = knot_data["Equivalent Braids"].apply(eval)
knot_data["Jones"] = knot_data["Jones"].apply(eval)

In [34]:
eq = knot_data["Equivalent Braids"]

maxCrossings = max([max([len(braid) for braid in braids]) for braids in eq])
maxStrands = max([max([max([abs(b) for b in braid]) for braid in braids]) for braids in eq])

minPower = -13
maxPower = 16
nCoefficients = maxPower - minPower + 1

In [35]:
# split dataset for training

train_split = .8

train_data = knot_data.sample(frac=train_split)
test_data = knot_data.drop(train_data.index)

x_train, y_train = parse_equivalent_braids(train_data["Equivalent Braids"],
                    train_data["Jones"], maxStrands, maxCrossings)

x_test, y_test = parse_equivalent_braids(test_data["Equivalent Braids"],
                    test_data["Jones"], maxStrands, maxCrossings)

In [45]:
# build model from functional api

inputs = keras.Input(shape=(maxCrossings, maxStrands*2))

x = layers.Masking()(inputs)
x = layers.GRU(64)(x)

outputs = layers.Dense(nCoefficients)(x)

In [46]:
model = keras.Model(inputs=inputs, outputs=outputs, 
    name="jones_polynomial")

model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mean_squared_error"])

model.summary()

Model: "jones_polynomial"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 19, 12)]          0         
                                                                 
 masking_3 (Masking)         (None, 19, 12)            0         
                                                                 
 gru_5 (GRU)                 (None, 64)                14976     
                                                                 
 dense_5 (Dense)             (None, 30)                1950      
                                                                 
Total params: 16,926
Trainable params: 16,926
Non-trainable params: 0
_________________________________________________________________


In [47]:
history = model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=5
)

Epoch 1/5
270/595 [============>.................] - ETA: 3s - loss: 75.7966 - mean_squared_error: 75.7966

KeyboardInterrupt: 

In [26]:
model.evaluate(x_test, y_test)

595/595 [==============================] - 1s 1ms/step - loss: 65.0339 - mean_squared_error: 65.0339


[65.0339126586914, 65.0339126586914]

In [27]:
def predict_coefficients(braid, maxStrands, maxCrossings):
    return np.asarray(model(np.array([encode_braid(braid, maxStrands, maxCrossings)])))[0]

In [28]:
braid = test_data.iloc[10]
word = braid["Equivalent Braids"][0]
jones = braid["Jones"]

prediction = predict_coefficients(word, maxStrands, maxCrossings)

In [29]:
for i in range(len(jones)):
    print(jones[i], round(prediction[i]), round(prediction[i] - jones[i]))

0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 1 1
0 -1 -1
0 1 1
0 0 0
0 0 0
0 1 1
0 -1 -1
1 2 1
-1 -2 -1
2 3 1
-2 -3 -1
2 5 3
-2 -5 -3
2 6 4
-1 -5 -4
0 4 4
0 -3 -3
0 1 1
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
